# Final Project: Data Science Salary Analysis
By: Anyah Luna

**Part 1:** Clean the Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from collections import defaultdict

In [10]:
# Load the full original dataset
from google.colab import files
uploaded = files.upload()


Saving kaggle_survey_2022_responses.csv to kaggle_survey_2022_responses (2).csv


In [22]:
df = pd.read_csv('kaggle_survey_2022_responses.csv')
df = df.iloc[1:].copy()


<ipython-input-22-b750f5a1133f>:1: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,208,215,225,248,255,257,260,270,271,272,277,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('kaggle_survey_2022_responses.csv')


In [24]:
#age to numeric avg
def age_to_mid(x):
    if pd.isna(x): return np.nan
    x = str(x).strip()
    if '+' in x: return int(x.replace('+','')) + 2
    try:
        low, high = map(int, x.split('-'))
        return (low + high) / 2
    except:
        return np.nan

df['Age_mid'] = df['Q2'].apply(age_to_mid)

In [25]:
# education to ordinal
edu_levels = [
    "No formal education past high school",
    "Some college/university study without earning a bachelor’s degree",
    "Bachelor’s degree",
    "Master’s degree",
    "Doctoral degree",
    "Professional doctorate"
]
edu_map = {level: i for i, level in enumerate(edu_levels)}
df['Education_ord'] = df['Q8'].map(edu_map)

#experience to numeric
experience_map = {
    '< 1 years': 0.5,
    '1-2 years': 1.5,
    '3-5 years': 4,
    '5-10 years': 7.5,
    '10-20 years': 15,
    '20+ years': 25
}
df['YearsCode_num'] = df['Q11'].map(experience_map)


In [27]:
#create other countries
country_counts = df['Q4'].value_counts()
rare_countries = country_counts[country_counts < 50].index
df['Country_clean'] = df['Q4'].replace(rare_countries, 'Other')


In [29]:
# Identify all multi-select columns
multi_part_cols = [col for col in df.columns if any(col.startswith(f"Q{n}_") for n in range(6, 45))]
from collections import defaultdict

# Group by question
grouped_by_question = defaultdict(list)
for col in multi_part_cols:
    question = col.split('_')[0]
    grouped_by_question[question].append(col)

# Create binary columns
for question, cols in grouped_by_question.items():
    for col in cols:
        values = df[col].dropna().unique()
        for val in values:
            if pd.isna(val): continue
            clean_name = f"{question}_{val}".strip().replace(" ", "_").replace("/", "_").replace("-", "_").replace(".", "_").replace("(", "").replace(")", "")
            df[clean_name] = (df[col] == val).astype(int)

# Drop original multi-select columns
df.drop(columns=multi_part_cols, inplace=True)


<ipython-input-29-f7463e5f96ca>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[clean_name] = (df[col] == val).astype(int)
<ipython-input-29-f7463e5f96ca>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[clean_name] = (df[col] == val).astype(int)
<ipython-input-29-f7463e5f96ca>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [26]:
# average sal
def salary_to_mid(x):
    if pd.isna(x) or 'Prefer not' in x or 'I do not wish to disclose' in x:
        return np.nan
    x = x.replace('$','').replace(',','').replace('+','').strip()
    if 'or more' in x:
        try:
            return float(x.split(' ')[0])
        except:
            return np.nan
    try:
        low, high = map(float, x.split('-'))
        return (low + high) / 2
    except:
        return np.nan

df['Salary'] = df['Q29'].apply(salary_to_mid)

In [54]:
# One-hot encode only truly unprocessed single-answer categoricals
leftover_categoricals = ['Q22', 'Q23', 'Q24', 'Q27']
leftover_categoricals = [col for col in leftover_categoricals if col in df.columns]

df = pd.get_dummies(df, columns=leftover_categoricals, prefix=leftover_categoricals)

In [64]:
#q16
ml_exp_map = {
    'I do not use machine learning methods': 0,
    'Under 1 year': 0.5,
    '1-2 years': 1.5,
    '2-3 years': 2.5,
    '3-4 years': 3.5,
    '4-5 years': 4.5,
    '5-10 years': 7.5,
    '10-20 years': 15,
    '20 or more years': 25
}

if 'Q16' in df.columns:
    df['ML_Experience_yrs'] = df['Q16'].map(ml_exp_map)
    df['ML_Experience_yrs'] = df['ML_Experience_yrs'].fillna(df['ML_Experience_yrs'].median())
    df.drop(columns=['Q16'], inplace=True)

# Q26: Map Team Size
team_size_map = {
    '0': 0,
    '1-2': 1,
    '3-4': 2,
    '5-9': 3,
    '10-14': 4,
    '15-19': 5,
    '20+': 6,
    'I do not know': -1
}


# q 27
if 'Q27' in df.columns:
    df = pd.get_dummies(df, columns=['Q27'], prefix='ML_Use')

# Map Q30 to numeric midpoints
spend_map = {
    '$0 ($USD)': 0,
    '$1-$99': 50,
    '$100-$999': 550,
    '$1000-$9,999': 5500,
    '$10,000-$99,999': 55000,
    '$100,000 or more ($USD)': 100000
}

if 'Q30' in df.columns:
    df['ML_Spend_5yr'] = df['Q30'].map(spend_map)
    df['ML_Spend_5yr'] = df['ML_Spend_5yr'].fillna(df['ML_Spend_5yr'].median())

    # Optional: drop Q30 after mapping
    df.drop(columns=['Q30'], inplace=True)



In [65]:
# Fill missing experience values with median (continuous variable)
if 'YearsCode_num' in df.columns:
    df['YearsCode_num'] = df['YearsCode_num'].astype(float).fillna(df['YearsCode_num'].median())

# Fill missing company size with median ordinal value
if 'CompanySize_ord' in df.columns:
    df['CompanySize_ord'] = df['CompanySize_ord'].astype(float).fillna(df['CompanySize_ord'].median())

# Fill missing team size with median ordinal value
if 'TeamSize_ord' in df.columns:
    df['TeamSize_ord'] = df['TeamSize_ord'].astype(float).fillna(df['TeamSize_ord'].median())

# Fill missing edu with median
if 'Education_ord' in df.columns:
    df['Education_ord'] = df['Education_ord'].astype(float).fillna(df['Education_ord'].median())


In [66]:
# Define raw columns to remove
raw_columns_to_drop = [
    'Duration (in seconds)', 'Q2', 'Q3', 'Q4', 'Q5', 'Q8', 'Q9', 'Q11', 'Q29', 'Q32', 'Q43',
    'Q10_1', 'Q10_2', 'Q10_3',
    'Q16', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q30'
]

# Add multi-part columns
multi_part_prefixes = [f"Q{i}_" for i in range(6, 45)]
multi_part_columns = [col for col in df.columns if any(col.startswith(prefix) for prefix in multi_part_prefixes)]

# Combine and drop
all_raw_columns = raw_columns_to_drop + multi_part_columns
df.drop(columns=[col for col in all_raw_columns if col in df.columns], inplace=True)


In [69]:
#fix team size was not mapping
df_raw = pd.read_csv("kaggle_survey_2022_responses.csv", low_memory=False).iloc[1:].copy()
df['Q26'] = df_raw['Q26']

team_size_map = {
    '0': 0,
    '1-2': 1,
    '3-4': 2,
    '5-9': 3,
    '10-14': 4,
    '15-19': 5,
    '20+': 6,
    'I do not know': -1
}

df['TeamSize_ord'] = df['Q26'].map(team_size_map)
df['TeamSize_ord'] = df['TeamSize_ord'].fillna(df['TeamSize_ord'].median())

In [72]:
# redrop
df.drop(columns=['Q26'], inplace=True)


In [73]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor

# Features to use for prediction — using only those present
core_features = ['Education_ord', 'YearsCode_num', 'Age_mid', 'CompanySize_ord', 'TeamSize_ord', 'Country_clean']
core_features = [col for col in core_features if col in df.columns]

# Prepare known and unknown salary subsets
model_data = df[core_features + ['Salary']]
known_salary = model_data[model_data['Salary'].notnull()]
missing_salary = model_data[model_data['Salary'].isnull()]

X_known = known_salary[core_features]
y_known = known_salary['Salary']
X_missing = missing_salary[core_features]

# Build preprocessing pipeline
numeric_features = [col for col in core_features if col != 'Country_clean']
categorical_features = ['Country_clean'] if 'Country_clean' in core_features else []

preprocessor = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='median'), numeric_features),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
])

# Full pipeline with Gradient Boosting Regressor
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

# Train and predict
if not X_missing.empty:
    pipe.fit(X_known, y_known)
    df.loc[X_missing.index, 'Salary'] = pipe.predict(X_missing)


In [74]:
df.to_csv("kaggle_survey_cleaned_final.csv", index=False)
from google.colab import files
files.download("kaggle_survey_cleaned_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>